In [59]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
#Scrape news site
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [5]:
html = browser.html
soup = bs(html, "html.parser")

In [13]:
#Scrape first news title
news_title = soup.find_all("div", class_="content_title")
first_news_title = news_title[0].text

#Scrape news p
news_p = soup.find_all("div", class_="article_teaser_body")
first_news_p = news_p[0].text
print(f"Title: {first_news_title}")
print(f"P: {first_news_p}")

Title: Things Are Stacking up for NASA's Mars 2020 Spacecraft
P: As the July 2020 launch date inches closer, the next spacecraft headed to the Red Planet is assembled for more testing.


In [183]:
#Scrape featured image
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [184]:
#Search for image
featured_image= browser.find_by_css(".carousel_item")
print(featured_image[0].html)
#image_html = featured_image[0].html


			  <div class="default floating_text_area ms-layer">
				<h2 class="category_title">
				  
				</h2>
				<h2 class="brand_title">
				  FEATURED IMAGE
				</h2>
				<h1 class="media_feature_title">
				  President's Signature Onboard Curiosity				</h1>
				<div class="description">
				  
				</div>
				<footer>
				  <a class="button fancybox" data-description="This view of Curiosity's deck shows a plaque bearing several signatures of US officials, including that of President Obama and Vice President Biden. The image was taken by the rover's Mars Hand Lens Imager (MAHLI)." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA15883_ip.jpg" data-link="/spaceimages/details.php?id=PIA15883" data-title="President's Signature Onboard Curiosity" id="full_image">
					FULL IMAGE
				  </a>
				</footer>
			  </div>
			  <div class="gradient_container_top"></div>
			  <div class="gradient_container_bottom"></div>
			


In [185]:
#Get link to feature image largesize
#image_soup = bs(image_html, "html.parser")
image_soup = bs(browser.html, "html.parser")
#image = image_soup.a.get("data-fancybox-href")
#image = image_soup.a.get("data-link")
image = image_soup.find("a", class_="fancybox")
#image = image_soup.find(id="full_image")
featured_image_url = f"https://www.jpl.nasa.gov{image['data-link']}"
#image['data-link']
#Visit page with largesize of featured image
browser.visit(featured_image_url)
feature_html = browser.html
feature_soup = bs(feature_html, "html.parser")

In [186]:
#Get largesize of featured image
feature_image = feature_soup.find("img", class_="main_image")
image = feature_image['src']
featured_image_url = f"https://www.jpl.nasa.gov{image}"
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA15883_hires.jpg


In [33]:
#Scrape weather from tweet
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

In [34]:
tweet_html = browser.html
tweet_soup = bs(tweet_html, "html.parser")

In [50]:
tweet = tweet_soup.find_all("p", class_="tweet-text")
mars_weather = tweet[0].text
print(mars_weather)

InSight sol 138 (2019-04-17) low -97.7ºC (-143.9ºF) high -17.3ºC (0.9ºF)
winds from the W at 4.3 m/s (9.5 mph) gusting to 12.6 m/s (28.1 mph)
pressure at 7.30 hPapic.twitter.com/ofMyPZM2vS


In [164]:
#Remove twitter pic
new_mars_weather = mars_weather.split("pic")
print(new_mars_weather[0])

InSight sol 138 (2019-04-17) low -97.7ºC (-143.9ºF) high -17.3ºC (0.9ºF)
winds from the W at 4.3 m/s (9.5 mph) gusting to 12.6 m/s (28.1 mph)
pressure at 7.30 hPa


In [73]:
#Scrape Mars facts
url = "https://space-facts.com/mars/"
browser.visit(url)

In [74]:
facts_html = browser.html
facts_soup = bs(facts_html, "html.parser")

In [87]:
facts = facts_soup.find("table", class_="tablepress-id-mars")
#convert tag to str so it can be read
facts_table = pd.read_html(str(facts))
#facts_table_rename = facts_table.rename(columns={"0":"Description", "1":"Value"})
#type(facts)
facts_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [94]:
#Get df from list and rename columns
facts_df = facts_table[0]
facts_df_rename = facts_df.rename(columns={0:'Description',1:'Value'})

#set new index
final_facts_table= facts_df_rename.set_index("Description")
final_facts_table

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [95]:
facts_html_table = final_facts_table.to_html()
facts_html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

In [98]:
#Remove \n from table
clean_facts_html_table= facts_html_table.replace('\n', '')
clean_facts_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [99]:
clean_facts_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [100]:
#Scrape hemispheres
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [134]:
#hemispheres=[]
hemisphere_image_urls =[]
#names = ["Cerberus","Schiaparelli","Syrtis","Valles"]
names = ["Cerberus Hemisphere","Schiaparelli Hemisphere","Syrtis Major Hemisphere","Valles Marineris Hemisphere"]
for name in names:
    browser.click_link_by_partial_text(name)
    hemisphere_html = browser.html
    hemisphere_soup = bs(hemisphere_html, "html.parser")
    hemisphere_image = hemisphere_soup.find("img", class_="wide-image")
    #hemisphere_image['src']
    #hemispheres.append(f"https://astrogeology.usgs.gov{hemisphere_image['src']}")
    hemisphere = f"https://astrogeology.usgs.gov{hemisphere_image['src']}"
    hemisphere_image_urls.append({"title":name,"img_url":hemisphere})
    browser.visit(url)

In [120]:
# hemisphere_image = hemisphere_soup.find("img", class_="wide-image")
# hemisphere_image['src']
# hemispheres.append(f"https://astrogeology.usgs.gov{hemisphere_image['src']}")

In [144]:
for image in hemisphere_image_urls:
    print(image['img_url'])

https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg
